In [111]:
import numpy as np
import matplotlib.pylab as pylab
# %matplotlib inline

In [112]:
def ellipsoid(params, shape=None, out=None):
    """
    Generate a rectangle containing an ellipse defined by its parameters.
    If out is given, fills the given cube instead of creating a new one.
    """
    # handle inputs
    if shape is None and out is None:
        raise ValueError("You need to set shape or out")
    if out is None:
        out = np.zeros(shape)
    if shape is None:
        shape = out.shape
    if len(shape) == 1:
        shape = shape, shape
    elif len(shape) > 2:
        raise ValueError("input shape must be lower or equal to 2")
    
    # define coords
    coords = define_coords(shape)

    # rotate coords
    coords = transform(coords, params)

    # recast as ndarray
    coords = [np.asarray(u) for u in coords]

    # reshape coords
    x, y = coords
    x.resize(shape)
    y.resize(shape)

    # fill ellipsoid with value
    out[(x ** 2 + y ** 2) <= 1.] += params['mag']
    return out

def rotation_matrix(p):
    """
    Defines a rotation matrix from angle phi.
    """
    cphi = np.cos(np.radians(p['phi']))
    sphi = np.sin(np.radians(p['phi']))
    alpha = [[cphi, sphi], [-sphi, cphi]]
    return np.asarray(alpha)

def transform(coords, p):
    """
    Apply rotation and translation to a 2-tuple of coords.
    """
    alpha = rotation_matrix(p)
    x, y = coords
    ndim = len(coords)
    out_coords = [sum([alpha[j, i] * coords[i] for i in range(ndim)])
                  for j in range(ndim)]
    M0 = [p['x'], p['y']]
    sc = [p['a'], p['b']]
    out_coords = [(u - u0) / su for u, u0, su in zip(out_coords, M0, sc)]
    return out_coords

def define_coords(shape):
    """
    Generate a tuple of coords in 2D with a given shape.
    """
    cshape = np.asarray(1j) * shape
    x, y = np.mgrid[-1:1:cshape[0], -1:1:cshape[1]]
    return x, y

In [114]:
def generate(lst, t):
    shape_list = []
    par_list = []
    for l in lst:
        if l['op']=='define':
            e = l.copy()
            shape_list.append(e)
            pard = {}
            for key in l:
                if not (key=='op' or key=='id' or key=='shape'):
                    pard[key] = np.ones(100)*l[key]
            par_list.append(pard)
        elif l['op']=='adjust':
            cl = par_list[l['id']][l['var']]
            cl[l['t_start']:l['t_end']] = np.linspace(cl[l['t_start']],l['new_val'],l['t_end']-l['t_start'])
            cl[l['t_end']-1:] = l['new_val']
    imgs = []
    for i in range(100):
        for key in ['a','b','phi','x','y']:
            shape_list[0][key] = par_list[0][key][i]
        imgs.append(ellipsoid(shape_list[0], shape=(256,256)))
    return imgs

In [118]:
def_operation = {"op":"define", "id":0, "shape":"ellipse", "x":0,"y":0, "a":0.1, "b":0.2, "phi":0, "mag":1}
shiftx = {"op":"adjust", "id":0,"var":"x", "new_val":1, "function":"linear", "t_start":0, "t_end":100}
shifty = {"op":"adjust", "id":0,"var":"y", "new_val":0.4, "function":"linear", "t_start":0, "t_end":100}
rot = {"op":"adjust", "id":0,"var":"phi", "new_val":80, "function":"linear", "t_start":0, "t_end":100}
appear1 = {"op":"adjust", "id":0,"var":"a", "new_val":0.3, "function":"linear", "t_start":10, "t_end":90}
appear2 = {"op":"adjust", "id":0,"var":"b", "new_val":0.2, "function":"linear", "t_start":10, "t_end":90}


import json
jsonstr = json.dumps([def_operation, shiftx, shifty, appear1, appear2])

In [119]:
lst = json.loads(jsonstr)
imgs = generate(lst, 1)

In [120]:
import scipy.misc as sm
for i, img in enumerate(imgs):
    sm.imsave('/tmp/img{:04}.png'.format(i),img)